In [ ]:
import os

In [ ]:
os.chdir("../")

In [ ]:
import os
from fastapi import FastAPI, HTTPException, UploadFile
from fastapi.responses import FileResponse, JSONResponse, PlainTextResponse
from pydantic import BaseModel
import uvicorn
import subprocess

from git import Repo as GitRepo, GitCommandError
from dvc.repo import Repo as DVCRepo

from src.Classification.utils.common import read_yaml, write_yaml, get_local_version, update_number_local_version
from src.Classification.constants import *
from src.Classification import logger
from box import ConfigBox

In [ ]:
git_repo = GitRepo(".")
dvc_repo = DVCRepo(".")

In [ ]:
ImageSizes = [[40, 40, 3], [80, 80, 3]]
DatasetRatios = [[0.8, 0.1, 0.1], [0.6, 0.2, 0.2]]
LearningRates = [0.0001, 0.00001]
BatchSizes = [16, 32]
Epochs = [10, 20]

In [ ]:
BRANCH_NAME = "tgv"
TMP_BRANCH_NAME = "tmp"
params = read_yaml(PARAMS_FILE_PATH)
i = 0

In [ ]:
for image_size in ImageSizes:
    for dataset_ratio in DatasetRatios:
        for learning_rate in LearningRates:
            for batch_size in BatchSizes:
                for epoch in Epochs:
                    i += 1
                    print(f"Running experiment {i}")
                    params.IMAGE_SIZE = image_size
                    params.TRAIN_VAL_TEST_RATIO = dataset_ratio
                    params.LEARNING_RATE = learning_rate
                    params.BATCH_SIZE = batch_size
                    params.EPOCHS = epoch
                    write_yaml(PARAMS_FILE_PATH, params)

                    dvc_repo.reproduce()
                    print("Reproduce done")
                    dvc_repo.push()
                    print("Reproduce pushed")

                    update_number_local_version(type="patch")
                    version_name = get_local_version()

                    git_repo.git.checkout("-b", TMP_BRANCH_NAME)
                    git_repo.git.add(A=True)
                    git_repo.index.commit(f"Experiment {i}")
                    git_repo.git.tag(version_name)
                    git_repo.git.checkout(BRANCH_NAME)
                    git_repo.git.branch("-D", TMP_BRANCH_NAME)
                    git_repo.git.checkout(version_name)
                    dvc_repo.pull(force=True)
                    print("Experiment done")
                    print("----------------------------------")
